# Investigating Options 
-- Nikhil Ramchandani
   


## Methods
#### - Binomial Option Price Modelling
#### - Black Sholes Model


In [37]:
#Dependencies
from yahoo_fin import options as op
from yahoo_fin import stock_info as si
import yfinance as yf
import numpy as np
import pandas as pd
import scipy.stats as ss
from datetime import datetime, date
import math

In [38]:
#Which stock options we are looking at
ticker = 'AAPL' 


In [39]:
#date is a list of expiration dates
#We are getting all the data of both call and put options of the given date below
dates = op.get_expiration_dates(ticker)
date = dates[12]
call = op.get_calls(ticker, date)
spot = si.get_live_price(ticker)
date = datetime.strptime(date, '%B %d, %Y').date()
date




datetime.date(2024, 4, 19)

In [40]:
#^TNX 10-year U.S. treasury yield, which will be used as our risk free interest rate
r = si.get_live_price('^TNX') / 100.0
r 
r = 0.04

In [41]:
# To calculate t (time till maturity in years), we will need todays date: 
today = date.today()
t = (date - today).days/365
t = float(t)
t

0.6136986301369863

In [42]:
# To calculate the volatility we will take the standard deviation of the returns over the past calendar year
stock_data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
closing_prices = stock_data["Close"]
returns = closing_prices.pct_change().dropna()
volatility = np.std(returns)
volatility

[*********************100%%**********************]  1 of 1 completed


0.02242599876541753

In [43]:
call.head()



,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL240419C00080000,2023-08-22 11:28AM EDT,80.0,100.47,100.80,101.90,0.0,-,2,4,74.65%
1,AAPL240419C00085000,2023-08-17 10:10AM EDT,85.0,93.16,96.35,97.35,0.0,-,-,1,73.03%
2,AAPL240419C00100000,2023-08-24 2:36PM EDT,100.0,81.40,81.85,83.30,0.0,-,2,4,63.21%
3,AAPL240419C00110000,2023-08-24 11:49AM EDT,110.0,72.05,72.65,73.65,0.0,-,4,4,57.35%
4,AAPL240419C00115000,2023-08-22 11:28AM EDT,115.0,67.47,67.75,68.85,0.0,-,-,1,53.91%


## 1. Binomial Options Price Modelling

Binomial Option Pricing is based off the theory that a stock price can only go in 2 directions, up or down. Expected payoffs are calculated by splitting time into descrete steps. The model uses risk-neutral probabilities to estimate option values through an iterative process using a binomial tree structure

As we look at stock only up or down we use u and d to define the potential values of the asset after one iteration.


We will be using the volatility of the stock to calculate u and d.

$$ u = e^{\sigma \sqrt{\Delta t}} $$

$$ d = e^{-\sigma \sqrt{\Delta t}} $$

Where

$$ u = up $$
$$ d = down $$
$$ \sigma = Volatility $$
$$ \Delta t = Time \space Length \space of \space Time \space Step$$ 

As mentioned above we will be using risk-neutral probabilities. The equation is given below.

$$ p = \frac{e^{r\Delta t} - d}{u - d}$$


Where

$$ u = risk-neutral \space probability$$
$$ u = up $$
$$ d = down $$
$$ \Delta t = Time \space length \space of \space time \space step$$ 

We will be looking at the impact of taking N number of steps in the binomial option model. The time till maurity of the asset will be divided by N, giving us the time step.

$$ \Delta t = \frac {t}{N} $$

Where

$$ t = Time \space Till \space Maturity \space (Years)$$ 
$$ N = Steps $$
$$ \Delta t = Time \space length \space of \space time \space step$$ 


We will be looking at the following number of steps, N = [1,10, 100, 500, 1000]

In [44]:
#Function which performs binomial option pricing with N steps
N = [1, 10, 100, 1000]
def binom_call(s, k, sigma, r, t, N):
    prices = []
    for i in N:
        step_size = t/i
        u = np.exp(sigma * np.sqrt(step_size))
        d = np.exp(-sigma * np.sqrt(step_size))
        p = (np.exp(r*step_size) - d)  /  (u - d)
        price = 0 
        for step in range(i+1):
            prob = math.comb(i, step)*p**step*(1-p)**(i-step)
            st = s*(u)**step*(d)**(i-step)
            price += max(st-k,0) * prob
        prices.append(price)
    return prices
            
        

In [45]:
#Function which loops through the options and call the binom_call function
def binom_loop(options):
    binom = []
    for index, option in options.iterrows():
        strike = option["Strike"]
        binom.append(binom_call(spot, strike, volatility, r, t, N))
    final_result = [[] for _ in range(4)]
    for sub_array in binom:
        for i, element in enumerate(sub_array):
            final_result[i].append(element)
    call["Binom Price 1 Step"] = final_result[0]
    call["Binom Price 10 Step"] = final_result[1]
    call["Binom Price 100 Step"] = final_result[2]
    call["Binom Price 1000 Step"] = final_result[3]
    
    

In [46]:
binom_loop(call)

In [47]:
x = ['Contract Name', 'Strike', 'Last Price', 'Binom Price 1 Step', 'Binom Price 10 Step', 'Binom Price 100 Step', 'Binom Price 1000 Step']
call[x].head()

,Contract Name,Strike,Last Price,Binom Price 1 Step,Binom Price 10 Step,Binom Price 100 Step,Binom Price 1000 Step
0,AAPL240419C00080000,80.0,100.47,103.120405,103.120405,103.120405,103.120405
1,AAPL240419C00085000,85.0,93.16,98.120405,98.120405,98.120405,98.120405
2,AAPL240419C00100000,100.0,81.40,83.120405,83.120405,83.120405,83.120405
3,AAPL240419C00110000,110.0,72.05,73.120405,73.120405,73.120405,73.120405
4,AAPL240419C00115000,115.0,67.47,68.120405,68.120405,68.120405,68.120405


As the number of steps N approaches infinity it converges to the Black Scholes model which is what we will investigate below

## 2. Black Scholes Model

Below is the general call option variant of the black scholes (BSM) equation:

$$ C = S_0 N(d_1) - K e^{-rt} N(d_2) $$

Where

$$ d_1 = \frac{1}{\sigma \sqrt{t}} \biggl[ \ln \biggl( \frac{S_0}{K} \biggr) + \biggl(r + \frac{\sigma^2}{2} \biggr) t \biggr] \quad \mbox{and} \quad d_2 = d_1 - \sigma \sqrt{t} $$

And

$$ C = Cost \space of \space Call \space Option $$
$$ S_0 = Spot \space Price $$
$$ K = Strike \space Price $$
$$ \sigma = Volatility $$
$$ r = Risk \space Free \space Interest $$
$$ t = Time \space Till \space Maturity \space (Years)$$ 


In [48]:
# Function that takes in required parameters to use BSM call equation
# Python equivalence to BSM equation above
def bsm_call(s, k, sigma, r, t):
    d1 = (np.log(s/k)+(r + (sigma**2)/2)*t)/(sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    return ss.norm.cdf(d1) * s - ss.norm.cdf(d2)* k* math.exp(-r * t)
    
    
    

In [49]:
#Function which loops though all the available options and use the bms_call function to calculate each option price

def bsm_loop(options):
    bsm = []
    for index, option in options.iterrows():
        strike = option["Strike"]
        bsm.append(bsm_call(spot, strike, volatility, r, t))
    call["BSM Prices"] = bsm


In [50]:
bsm_loop(call)

In [51]:
call.columns

Index(['Contract Name', 'Last Trade Date', 'Strike', 'Last Price', 'Bid',
       'Ask', 'Change', '% Change', 'Volume', 'Open Interest',
       'Implied Volatility', 'Binom Price 1 Step', 'Binom Price 10 Step',
       'Binom Price 100 Step', 'Binom Price 1000 Step', 'BSM Prices'],
      dtype='object')

In [52]:
x = ['Contract Name', 'Strike', 'Last Price', 'BSM Prices']
call[x].head()

,Contract Name,Strike,Last Price,BSM Prices
0,AAPL240419C00080000,80.0,100.47,100.619829
1,AAPL240419C00085000,85.0,93.16,95.741074
2,AAPL240419C00100000,100.0,81.40,81.104811
3,AAPL240419C00110000,110.0,72.05,71.347302
4,AAPL240419C00115000,115.0,67.47,66.468547


In [53]:
call

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,Binom Price 1 Step,Binom Price 10 Step,Binom Price 100 Step,Binom Price 1000 Step,BSM Prices
0,AAPL240419C00080000,2023-08-22 11:28AM EDT,80.0,100.47,100.80,101.90,0.00,-,2,4,74.65%,103.120405,103.120405,1.031204e+02,1.031204e+02,1.006198e+02
1,AAPL240419C00085000,2023-08-17 10:10AM EDT,85.0,93.16,96.35,97.35,0.00,-,-,1,73.03%,98.120405,98.120405,9.812041e+01,9.812041e+01,9.574107e+01
2,AAPL240419C00100000,2023-08-24 2:36PM EDT,100.0,81.40,81.85,83.30,0.00,-,2,4,63.21%,83.120405,83.120405,8.312041e+01,8.312041e+01,8.110481e+01
3,AAPL240419C00110000,2023-08-24 11:49AM EDT,110.0,72.05,72.65,73.65,0.00,-,4,4,57.35%,73.120405,73.120405,7.312041e+01,7.312041e+01,7.134730e+01
4,AAPL240419C00115000,2023-08-22 11:28AM EDT,115.0,67.47,67.75,68.85,0.00,-,-,1,53.91%,68.120405,68.120405,6.812041e+01,6.812041e+01,6.646855e+01
5,AAPL240419C00125000,2023-09-06 9:48AM EDT,125.0,66.71,59.00,59.45,0.00,-,5,2,50.13%,58.120405,58.120405,5.812041e+01,5.812041e+01,5.671104e+01
6,AAPL240419C00130000,2023-08-29 3:07PM EDT,130.0,60.25,54.45,55.15,0.00,-,1,3,48.42%,53.120405,53.120405,5.312041e+01,5.312041e+01,5.183228e+01
7,AAPL240419C00135000,2023-09-08 1:15PM EDT,135.0,50.65,49.85,50.30,1.07,+2.16%,50,11,44.98%,48.120405,48.120405,4.812041e+01,4.812041e+01,4.695353e+01
8,AAPL240419C00140000,2023-09-08 9:55AM EDT,140.0,46.45,45.35,45.90,-9.45,-16.91%,2,60,42.80%,43.120405,43.120405,4.312041e+01,4.312041e+01,4.207477e+01
9,AAPL240419C00145000,2023-09-07 9:50AM EDT,145.0,40.62,41.40,41.55,0.00,-,1,247,40.62%,38.120405,38.120405,3.812041e+01,3.812041e+01,3.719602e+01


In [58]:
x = ['Contract Name', 'Strike', 'Last Price', 'BSM Prices', 'Binom Price 1 Step', 'Binom Price 1000 Step']
call[x]

,Contract Name,Strike,Last Price,BSM Prices,Binom Price 1 Step,Binom Price 1000 Step
0,AAPL240419C00080000,80.0,100.47,1.006198e+02,103.120405,1.031204e+02
1,AAPL240419C00085000,85.0,93.16,9.574107e+01,98.120405,9.812041e+01
2,AAPL240419C00100000,100.0,81.40,8.110481e+01,83.120405,8.312041e+01
3,AAPL240419C00110000,110.0,72.05,7.134730e+01,73.120405,7.312041e+01
4,AAPL240419C00115000,115.0,67.47,6.646855e+01,68.120405,6.812041e+01
5,AAPL240419C00125000,125.0,66.71,5.671104e+01,58.120405,5.812041e+01
6,AAPL240419C00130000,130.0,60.25,5.183228e+01,53.120405,5.312041e+01
7,AAPL240419C00135000,135.0,50.65,4.695353e+01,48.120405,4.812041e+01
8,AAPL240419C00140000,140.0,46.45,4.207477e+01,43.120405,4.312041e+01
9,AAPL240419C00145000,145.0,40.62,3.719602e+01,38.120405,3.812041e+01


In [69]:
#Calculating Absolute Percentage Errors (APE)
call['APE BSM'] = abs((call['Last Price'] - call['BSM Prices'])/call['Last Price'])
call['APE BIN 1'] = abs((call['Last Price'] - call['Binom Price 1 Step'])/call['Last Price'])
call['APE BIN 1000'] = abs((call['Last Price'] - call['Binom Price 1000 Step'])/call['Last Price'])
x = ['Contract Name', 'Strike', 'Last Price', 'APE BSM', 'APE BIN 1', 'APE BIN 1000']
call[x]

,Contract Name,Strike,Last Price,APE BSM,APE BIN 1,APE BIN 1000
0,AAPL240419C00080000,80.0,100.47,0.001491,0.026380,0.026380
1,AAPL240419C00085000,85.0,93.16,0.027706,0.053246,0.053246
2,AAPL240419C00100000,100.0,81.40,0.003626,0.021135,0.021135
3,AAPL240419C00110000,110.0,72.05,0.009753,0.014856,0.014856
4,AAPL240419C00115000,115.0,67.47,0.014843,0.009640,0.009640
5,AAPL240419C00125000,125.0,66.71,0.149887,0.128760,0.128760
6,AAPL240419C00130000,130.0,60.25,0.139713,0.118334,0.118334
7,AAPL240419C00135000,135.0,50.65,0.072981,0.049943,0.049943
8,AAPL240419C00140000,140.0,46.45,0.094192,0.071681,0.071681
9,AAPL240419C00145000,145.0,40.62,0.084293,0.061536,0.061536


In [76]:
#Mean Absolute Percentage Errors (MAPE)
print("BIN 1 MAPE:", call['APE BIN 1'].mean())
print("BIN 1000 MAPE:",call['APE BIN 1000'].mean())
print("BSM MAPE:", call['APE BSM'].mean())

BIN 1 MAPE: 0.6285103562126626
BIN 1000 MAPE: 0.6253437110715321
BSM MAPE: 0.6293968502089938


As shown above as we increase number of steps the MAPE decreases, signifying more accurate results. However the MAPE for all are high and would not suffice as prediction models. Both models seem to undervalue the options as we dcrease in value. The first few options of highest value seem to be predicted relatively well compared to the rest.